# ppg.ipynb
Functions to examine rolling correlations between device sensor outputs.

Authors:  
– Jon Clucas, 2017 jon.clucas@childmind.org  
– Arno Klein, 2017  
© 2017, Child Mind Institute, Apache v2.0 License

setup:

In [ ]:
%matplotlib inline
from annotate_range import annotation_line
from astropy.stats import median_absolute_deviation as mad
from chart_data import bland_altman_plot, df_devices, write_csv, xcorr
from chart_data_ppg import linechart
from config import short_dir, test_urls
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from normalize_acc_data import actigraph_acc, geneactiv_acc
from plot_normalized_vector_lengths import demean_and_normalize
from utilities.fetch_data import fetch_check_data, fetch_data, fetch_hash
import json, matplotlib as mpl, numpy as np, os, pandas as pd, matplotlib.pyplot as plt
"""
with open(os.path.join('./line_charts/device_colors.json')) as fp:
    color_key = json.load(fp)
"""
pd.set_option('mode.use_inf_as_null', True)
ppg_hashes = {'E4_ppg': '74a4074802c03fa22fb3ae8a6715263b',
              'Wavelet_ppg': '1af7572640aea22a0af354bedbeb7a1e'}
if not os.path.exists('./sample_data'):
    os.makedirs('./sample_data')

### PPG
E4 has a 'Timestamp' column and a combined red and green data column.
Wavelet has a 'Timestamp' column and four data columns: red, infrared, filtered red, and filtered infrared

In [ ]:
df = df_devices(['E4', 'Wavelet'], 'ppg', datetime(2017, 4, 7, 17), datetime(2017, 4, 7, 20), ppg_hashes)

Retrieve file from URL: https://osf.io/v3ad6/?action=download&version=2
Copy file to cache: sample_data/74a4074802c03fa22fb3ae8a6715263b/E4_ppg.csv


In [ ]:
print(df)

In [ ]:
[xcorr(df['nW'].values, df['infrared'].values), xcorr(df['nW'].values, df['red'].values),
 xcorr(df['infrared'].values, df['red'].values)]

In [ ]:
Evalues = df['nW'].values
WIvalues = df['infrared'].values
WRvalues = df['red'].values

shiftWIWR = len(WIvalues) - np.argmax(np.correlate(np.nan_to_num(WIvalues), np.nan_to_num(WRvalues), mode='full'))
shiftEWI = len(WIvalues) - np.argmax(np.correlate(np.nan_to_num(WIvalues), np.nan_to_num(Evalues), mode='full'))
shiftEWR = len(WRvalues) - np.argmax(np.correlate(np.nan_to_num(WRvalues), np.nan_to_num(Evalues), mode='full'))
shiftEW = np.int(np.nanmean([shiftEWI, shiftEWR]))
shift_EW = np.abs(shiftEW)
Evalues_shifted = Evalues[:Evalues.shape[0]-shift_EW]
WIvalues_shifted = WIvalues[shift_EW:WIvalues.shape[0]]
WRvalues_shifted = WRvalues[shift_EW:WRvalues.shape[0]]
[np.shape(Evalues_shifted), np.shape(WIvalues_shifted), np.shape(WRvalues_shifted)]
[xcorr(Evalues_shifted, WIvalues_shifted), xcorr(Evalues_shifted, WRvalues_shifted),
 xcorr(WIvalues_shifted, WRvalues_shifted)]

In [ ]:
linechart(demean_and_normalize(df), line=False)

In [ ]:
shifted_t = []
i = 0
while len(shifted_t) < np.shape(Evalues_shifted)[0]:
    shifted_t.append(df['nW'].index[i])
    i += 1
shifted_df = pd.DataFrame({'nW': Evalues_shifted,
            'infrared': WIvalues_shifted,
            'red': WRvalues_shifted, 'Timestamp':shifted_t})
shifted_df.set_index('Timestamp', inplace=True)
linechart(shifted_df, line=False)

In [ ]:
linechart(demean_and_normalize(shifted_df), line=False)

In [ ]:
df = df_devices(['E4', 'Wavelet'], 'ppg', datetime(2017, 4, 8, 15, 30), datetime(2017, 4, 8, 16), ppg_hashes)
[xcorr(df['nW'].values, df['infrared'].values), xcorr(df['nW'].values, df['red'].values),
 xcorr(df['infrared'].values, df['red'].values)]

In [ ]:
Evalues = df['nW'].values
WIvalues = df['infrared'].values
WRvalues = df['red'].values

shiftWIWR = len(WIvalues) - np.argmax(np.correlate(np.nan_to_num(WIvalues), np.nan_to_num(WRvalues), mode='full'))
shiftEWI = len(WIvalues) - np.argmax(np.correlate(np.nan_to_num(WIvalues), np.nan_to_num(Evalues), mode='full'))
shiftEWR = len(WRvalues) - np.argmax(np.correlate(np.nan_to_num(WRvalues), np.nan_to_num(Evalues), mode='full'))
shiftEW = np.int(np.nanmean([shiftEWI, shiftEWR]))
shift_EW = np.abs(shiftEW)
Evalues_shifted = Evalues[:Evalues.shape[0]-shift_EW]
WIvalues_shifted = WIvalues[shift_EW:WIvalues.shape[0]]
WRvalues_shifted = WRvalues[shift_EW:WRvalues.shape[0]]
[np.shape(Evalues_shifted), np.shape(WIvalues_shifted), np.shape(WRvalues_shifted)]
[xcorr(Evalues_shifted, WIvalues_shifted), xcorr(Evalues_shifted, WRvalues_shifted),
 xcorr(WIvalues_shifted, WRvalues_shifted)]

In [ ]:
linechart(demean_and_normalize(df), line=False)

In [ ]:
shifted_t = []
i = 0
while len(shifted_t) < np.shape(Evalues_shifted)[0]:
    shifted_t.append(df['nW'].index[i])
    i += 1
shifted_df = pd.DataFrame({'nW': Evalues_shifted,
            'infrared': WIvalues_shifted,
            'red': WRvalues_shifted, 'Timestamp':shifted_t})
shifted_df.set_index('Timestamp', inplace=True)
linechart(shifted_df, line=False)

In [ ]:
linechart(demean_and_normalize(shifted_df), line=False)

In [ ]:
df = df_devices(['E4'], 'ppg', datetime(2017, 4, 7, 17, 1), datetime(2017, 4, 7, 17, 6), ppg_hashes)
linechart(demean_and_normalize(df), line=False)

In [ ]:
df = df_devices(['Wavelet'], 'ppg', datetime(2017, 4, 7, 17), datetime(2017, 4, 7, 20), ppg_hashes)
linechart(demean_and_normalize(df), line=False)